In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-driver
!apt install -y chromium-chromedriver

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime
import re
import time

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    # options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    # driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver',options=options)
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
# reading csv
df=pd.read_csv("/content/drive/My Drive/data_2023-10-27_11-06-43.csv")

# changing data-types of columns to string
df['Source']=df['Source'].astype('string')
df['Published'] = df['Published'].astype('string')
df['Title'] = df['Title'].astype('string')
df['Link'] = df['Link'].astype('string')

df.head()

In [ ]:
# function to remove extra spaces from string
def remove_extra_spaces(text):
    new_text = re.sub(r'\s+', '+', text)
    new_text = new_text.strip()
    return new_text+'+news'

# function to trim string
def trim(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

# function to format date
def idate(date):
    mon={'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
    if date[-1]==' ':
        return date[-5:-1]+'-'+mon[date[2:5]]+'-0'+date[:1]
    return date[-4:]+'-'+mon[date[3:6]]+'-'+date[:2]

In [ ]:
# Creating Search Term(search_term) and Date(idate) columns having proper format to search and compare date on YouTube
# Search term is fromed by replacing space by '+' in title

search_term=[]
add_date=[]
for i in range(len(df)):
    search_term.append(remove_extra_spaces(df['Title'][i]))
    if len(df["Published"][i])==0:
        add_date.append("0000-00-00")
    elif df["Published"][i][4]=='-':
        add_date.append(df["Published"][i][:10])
    elif df["Published"][i][2]=='-':
        add_date.append(df["Published"][i][6:10]+'-'+df["Published"][i][3:5]+'-'+df["Published"][i][:2])
    else:
        add_date.append(idate(df["Published"][i][5:16]))
df['search_term']=search_term
df['idate']=add_date
df.head()

In [ ]:
#function to get date
def gdate(date):
    mon={'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}

    if len(date)==11:
        return date[-4:]+'-'+mon[date[:3]]+'-'+date[4:5]

    return date[-4:]+'-'+mon[date[:3]]+'-'+date[4:6]

#function to get views
def gviews(views):
    s=""
    for i in views:
        if i>='0' and i<='9':
            s+=i
    return int(s)

In [ ]:
# Controling browser from selinium
browser=web_driver()

In [ ]:
# Scrapping Views from Youtube

# Explanation of Method Used
# In this part we are searching top videos for title of articles and comparing their upload date with published date of article
# If upload date of video is within fews days(10 days) of publish date of article then that video is selected
# Max views from selected videos is considered and stored

views=[]                              # list to store views
for k in range(0,100):               # Iterating for first 100 rows

    v=-1
    browser.get(f"https://www.youtube.com/results?search_query={df['search_term'][k]}")          # Searching the title of article on youtube
    time.sleep(3)                                                                                # waiting to load the page
    ele = browser.find_elements(By.XPATH, '//div[@id="contents"]/ytd-video-renderer/div[@id="dismissible"]/ytd-thumbnail')  # Catching / Handleing videos from searchh results
    cnt=0

    for i in ele:    # Iterating the videos

        if cnt>=10:  # breaking at 10th video
            break
        cnt=cnt+1

        try:
            i.click()            # Cicking the videos
            time.sleep(1)        # waiting to load the video

            try:
                desc=browser.find_element(By.XPATH, '//div[@id="info-container"]/yt-formatted-string/span')  # Finding Description of video
                desc.click()
                time.sleep(1)

                pcont1=browser.find_elements(By.ID,"info-container")
                pcont2=pcont1[0].find_elements(By.ID,'info')
                pcont3=pcont2[0].find_elements(By.TAG_NAME,'span')

                viu=gviews(pcont3[0].text[:-6])                            # Scraping views of video
                dat1=gdate(pcont3[2].text)                                 # Scraping date of upload of video
                dat2=df["idate"][k]

                cdate1 = datetime.strptime(dat1, "%Y-%m-%d")
                cdate2 = datetime.strptime(dat2, "%Y-%m-%d")
                ddiff=cdate1-cdate2
                ddiff=ddiff.days

                if ddiff>=-10 and ddiff <=10 and viu>v:                   # Comparing date of upload of video with published date of article
                    v=viu

            except:
                pass

            browser.back()                                               # Back to previous page of searchh results
            time.sleep(1)

        except:
            pass

    views.append(v)
    print(k,v)

print(views)

In [ ]:
# Saving views to csv
# df2=pd.DataFrame(views)
# df2.to_csv('Yt_Views_Scrapping_4-4.5_csv',index=False)